In [ ]:
import catboost as ctb
import pandas as pd
import vertica_python
from vertica_python.errors import MissingRelation
import verticapy as vp
from verticapy.utilities import pandas_to_vertica, insert_into, drop, readSQL
import configparser

Helper functions

In [ ]:
config_path = 'config.ini'

def get_VRT_cursor(path=config_path):
    config = configparser.ConfigParser()
    config.read(path)
    
    credentials = {
        'database': config['VERTICA']['database'],
        'user': config['VERTICA']['user'],
        'password': config['VERTICA']['password'],
        'host': config['VERTICA']['host'],
        'port': config['VERTICA']['port']
    }
    
    conn = vertica_python.connect(**credentials)
    cursor = conn.cursor()
    return cursor, conn

def get_CH_cursor(path=config_path):
    config = configparser.ConfigParser()
    config.read(path)
    
    credentials = {
        'database': config['CLICKHOUSE']['database'],
        'user': config['CLICKHOUSE']['user'],
        'password': config['CLICKHOUSE']['password'],
        'host': config['CLICKHOUSE']['host'],
        'port': config['CLICKHOUSE']['port']
    }
    
    conn = Connection(**credentials)
    
    cursor = conn.cursor()
    return cursor

def get_df_from_sql(cursor, sql='SELECT now(), version()'):
    cursor.execute(sql)
    columns = cursor.description
    result = cursor.fetchall()
    df = pd.DataFrame(result, columns=[tuple[0] for tuple in columns])
    return df

def to_vertica(sql, table_name, CH_cursor, VRT_cursor, users_list):
    df = get_df_from_sql(CH_cursor, sql)
    
    try:
        readSQL(f"SELECT * FROM your_schema_name.{table_name}", limit = 1)
        drop(name = f"your_schema_name.{table_name}")
        print(f"Table your_schema_name.{table_name} droped.")
        
        pandas_to_vertica(df = df, name = table_name, schema = "your_schema_name")
    except MissingRelation:
        pandas_to_vertica(df = df, name = table_name, schema = "your_schema_name")
        
    VRT_cursor.execute(f"GRANT SELECT ON TABLE your_schema_name.{table_name} TO {', '.join(users_list)}")
    print(f"GRANT SELECT ON TABLE your_schema_name.{table_name} TO {', '.join(users_list)}")
    
def drop_table(table_name):
    try:
        readSQL(f"SELECT * FROM your_schema_name.{table_name}", limit = 1)
        drop(name = f"your_schema_name.{table_name}")
        print(f"Table your_schema_name.{table_name} droped.")
    except MissingRelation as e:
        print(str(e))    

CH_cursor = get_CH_cursor()
VRT_cursor, conn = get_VRT_cursor()
vp.set_connection(conn)

users_list = ['some_user']

'config.ini' file looks like this:
[CLICKHOUSE]
database = your_db_name 
user = your_username
password = your_password
host = your_host
port = your_port

[VERTICA]
database = your_db_name
user = your_username
password = your_password
host = your_host
port = your_port

Get data

In [ ]:
sql = """
SELECT *
FROM your_schema.your_table r
"""

In [ ]:
df = get_df_from_sql(cursor=VRT_cursor, sql=sql)

Load models

In [ ]:
model1 = ctb.CatBoostRegressor()      
model1.load_model('model_catboost_v1.cbm')
model2 = ctb.CatBoostRegressor()      
model2.load_model('model_catboost_v2.cbm')

In [ ]:
features = ['feature1', 'feature2', 'feature3', 'feature4']
cat_features = ['feature1', 'feature2']

In [ ]:
pool = ctb.Pool(df[features], cat_features = cat_features)

In [ ]:
y_predict1 = model1.predict(pool)
y_predict2 = model2.predict(pool)

In [ ]:
df['predict_old'] = y_predict1
df['predict_new'] = y_predict2
df

In [ ]:
df['accuracy_new'] = abs((df['predict_new'] - df['fact']) / df['fact'])
df['accuracy_old'] = abs((df['predict_old'] - df['fact']) / df['fact'])
df